In [ ]:
!pip install speechbrain

In [ ]:
import os
import sys
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
import speechbrain as sb
from speechbrain.pretrained import SpeakerRecognition
from speechbrain.utils.data_utils import download_file
from hyperpyyaml import load_hyperpyyaml
from speechbrain.utils.distributed import run_on_main
import pandas as pd
import numpy as np

In [ ]:
# подход распознавания на основе ECAPA: сравниваем свои голоса 
from speechbrain.pretrained import SpeakerRecognition
import os
from itertools import combinations
import torch


num_true=0
num_total=0

for root, dirs, files in os.walk('/Voices'):
    if not files:
        continue
    res = list(combinations(files, 2)) # получаем списки кортежей: [(файл 1, файл 2), (файл 1, файл 3)...(файл 3, файл 4)]
    

    for tuples in res:         # идем по каждому кортежу в списке
        verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")
        score, prediction = verification.verify_files(os.path.join(root, tuples[0]), os.path.join(root, tuples[1]))
        num_total+=1
        if prediction == torch.tensor([True]):
            num_true+=1

# считаем метрику accuracy:
if num_total !=0:
    accuracy = num_true/num_total
    print('accuracy: ', accuracy)
else:
    print('На ноль делить нельзя')

In [ ]:
# подход распознавания на основе ECAPA: сравниваем несвои голоса 
from speechbrain.pretrained import SpeakerRecognition
from itertools import product, chain
from os import walk
import torch

nums_true=0
nums_total=0

all_files = dict()
for root, dirs, files in walk('Voices'):
    if not files:
        continue
    all_files[root] = files

for root in all_files.copy():
    files = all_files[root]
    del all_files[root] 
    
    # chain(*iterables) возвращает по одному элементу из первого итератора, потом из второго, пока итераторы не кончатся
    # Пример: list(chain('ABCD', 'xy')) --> ['A', 'B', 'C', 'D', 'x', 'y']
    # собираем по одному файлу для каждого списка (в итоге получаем идентичные 12 списков)
    other_files = list(chain(*[all_files[key] for key in all_files]))

    # itertools.product(*iterables, repeat=1) - аналог вложенных циклов
    # Пример: product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    result = list(product(files, other_files))
   
    for pairs in result:
        verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-ecapa-voxceleb", savedir="pretrained_models/spkrec-ecapa-voxceleb")
        score, prediction = verification.verify_files(os.path.join(root, pairs[0]), os.path.join(root, pairs[1]))
        nums_total+=1
        if prediction == torch.tensor([True]):
            nums_true+=1

# считаем метрику accuracy:
if nums_total !=0:
    accuracy = nums_true/nums_total
    print(accuracy)
else:
    print('На ноль делить нельзя')